In [124]:
import pickle
import numpy as np

In [71]:
with open("nt_seq_dic.pkl", 'rb') as p:
    nt_seq_dic = pickle.load(p)

with open("gene_subtree_dic.pkl", 'rb') as p:
    gene_subtree_dic = pickle.load(p)

with open("total_BR_tree_dic.pkl", 'rb') as p:
    total_BR_tree_dic = pickle.load(p)

In [10]:
len(gene_subtree_dic.keys())

63

In [7]:
from mytree import *

In [76]:
import copy

count=0
gene_node_list = []
current_name = "Photosynthesis proteins"

for i in gene_subtree_dic.keys():
    count+=1
    tmp_node, _ = find_deepest_node(gene_subtree_dic[i], current_name)
    gene_node_list.append(tmp_node)


root = total_BR_tree_dic['ko00194']
print(cal_tree_distance(root, gene_node_list[-4], gene_node_list[-5]))

# clean up gene names
nice_name_gene_node_list = copy.deepcopy(gene_node_list)
for i in range(len(gene_node_list)):
    if i < 8:
        nice_name_gene_node_list[i].value = gene_node_list[i].value[16:].split(';')[0]
    else:
        nice_name_gene_node_list[i].value = gene_node_list[i].value[8:].split(';')[0]

4


In [118]:
flag={}
notin = set()
for i in nt_seq_dic.keys():
    for j in nice_name_gene_node_list:
        if i.lower() in j.value.lower():
            flag[i] = flag.get(i, 0)+1
            break
        elif j == nice_name_gene_node_list[-1]:
            notin.add(i)

In [112]:
correct = {
    'CYTC6A': 'petJ',
    'PSAE-2': 'psaE',
    'PSBP-1': 'psbP',
    'PnsL2': 'psbQ',
    'NPQ4': 'psbS',
    'PSAD-2': 'psaD',
    'PSAH2': 'psaH',
    'DRT112': 'petE',
    'FD1': 'petF',
    'FNR2':'petH'
}

In [116]:
# clean names in nice_name_gene_node_lise
for k,v in correct.items():
    for i in nice_name_gene_node_list:
        if v == i.value:
            i.value = k

In [119]:
notin

{'ATPC2', 'PSBO2', 'atpI'}

In [143]:
fixed_gene_names = list(nt_seq_dic.keys())
fixed_gene_names.remove('atpI')
fixed_gene_names.remove('PSBO2')
fixed_gene_names.remove('ATPC2')
only_names_gene_node = []

length = 0
for i in fixed_gene_names:
    for j in nice_name_gene_node_list:
        if i.lower() == j.value.lower():
            length+=1

for i in nice_name_gene_node_list:
    only_names_gene_node.append(i.value.lower())

print(length)

ground_truth_tree_dist = np.zeros((length, length), dtype=float)
for i in range(length):
    index = only_names_gene_node.index(fixed_gene_names[i].lower())
    node_i = gene_node_list[i]
    for j in range(length):
        index = only_names_gene_node.index(fixed_gene_names[j].lower())
        node_j = gene_node_list[j]
        ground_truth_tree_dist[i, j] = cal_tree_distance(root, node_i, node_j)

51


In [145]:
with open('ground_truth_dist_tuple.pkl', 'wb') as p:
    pickle.dump((fixed_gene_names, ground_truth_tree_dist), p)